In [2]:
import pandas as pd 
import numpy as np 

In [3]:
path_to_data = "~/downloads/vote.txt"

In [4]:
data = pd.read_csv(path_to_data , header = None)

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,0,1,0,1,1,0,0,1,1,0,0,1,1,0,1,1
1,0,0,1,1,1,1,1,1,0,1,0,0,0,1,0,1,0
2,1,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1,0
3,0,1,1,0,0,0,0,1,1,0,1,0,0,1,1,1,1
4,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,1


In [6]:
class_name = 16

In [ ]:
data = data.fillna("")

we write the main function which will calculate the numerator in the bayes formula and return the argmax

In [8]:
def nb_classifier(data_point, train_data, class_name):
    # Unique class values
    class_vals = train_data[class_name].unique()
    
    # Features (exclude class column)
    variables = [v for v in train_data.columns if v != class_name]
    
    # Log probabilities for each class
    log_prob = {}
    
    for c in class_vals:
        log_acc_prob = 0.0
        
        # Count total samples for the class
        count_class = train_data[train_data[class_name] == c].shape[0]
        
        # Prior probability of the class
        prior_class = count_class / train_data.shape[0]
        
        for v in variables:
            # Count occurrences of the feature value given the class
            count_x_and_class = train_data[
                (train_data[class_name] == c) & (train_data[v] == data_point[v])
            ].shape[0]
            
            # Handling zero probabilities (Laplace smoothing)
            if count_x_and_class == 0:
                smoothed_count = 1  # laplace constanst set to 1 
                smoothed_class_count = count_class + len(train_data[v].unique())
                prob = smoothed_count / smoothed_class_count
            else:
                prob = count_x_and_class / count_class
            
            # Accumulate log probabilities
            log_acc_prob += np.log(prob)
        
        #
        log_prob[c] = log_acc_prob + np.log(prior_class)
    
    # Return the class with the highest log probability 
    return max(log_prob, key=log_prob.get)


In [9]:
data_point = data.loc[3]
nb_classifier(data_point, data, class_name)

1

In [10]:
data = data.sample(frac=1 , random_state=42).reset_index(drop=True)

In [11]:

splits = [pd.DataFrame(s) for s in np.array_split(data.to_numpy(), 5, axis=0)]

In [12]:
split_1 = splits[0]


In [13]:
split_1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1,1
1,0,1,1,0,0,1,1,1,1,0,1,1,0,0,1,1,1
2,0,1,1,0,1,1,1,1,0,0,1,0,1,0,1,1,1
3,1,0,1,0,0,0,1,1,1,1,1,0,0,0,1,1,1
4,0,1,1,0,0,1,1,1,1,1,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,1
61,1,0,1,0,0,0,1,1,1,1,0,0,1,0,1,1,1
62,0,0,1,1,1,1,0,0,0,1,0,1,1,1,1,1,0
63,0,1,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0


In [14]:
import numpy as np

def cross_validate(nb_classifier, splits, class_name):
    errors = []  
    
    for i in range(len(splits)):
        # Use one fold as test set
        test_data = splits[i]
        
        # Combine other folds as training set
        train_data = pd.concat([splits[j] for j in range(len(splits)) if j != i], ignore_index=True)
        
        
        correct = 0
        total = len(test_data)
        
        for _, row in test_data.iterrows():
            # Predict the class for the test data point
            predicted_class = nb_classifier(row, train_data, class_name)
            actual_class = row[class_name]
            
            # Check if the prediction matches the actual class
            if predicted_class == actual_class:
                correct += 1
        
        # Calculate the error for this fold
        error = 1 - (correct / total)
        errors.append(error)
    
    # Return the average error across all folds
    return np.mean(errors)
average_error = cross_validate(nb_classifier, splits, class_name)
print(f"Average Error: {average_error}")


Average Error: 0.10153846153846154


In [15]:
accuracy = 1 - average_error
accuracy * 100

89.84615384615384

here we calculate the likelihood p ( x | class ) for the whole data as the question requested 

In [ ]:
def calculate_likelihood(data, class_name, feature_name):
    
    count_class = data[class_name].value_counts()
    
   
    count_feature_given_class = data.groupby(class_name)[feature_name].apply(lambda x: (x == 1).sum())  
    
 
    smoothed_prob = (count_feature_given_class + 1) / (count_class + 2)  
    
    return smoothed_prob


likelihoods = {}
for feature in data.columns:
    if feature != class_name:
        likelihoods[feature] = calculate_likelihood(data, class_name, feature)


print(likelihoods)

{0: 16
0    0.275862
1    0.591549
dtype: float64, 1: 16
0    0.560345
1    0.535211
dtype: float64, 2: 16
0    0.198276
1    0.863850
dtype: float64, 3: 16
0    0.974138
1    0.065728
dtype: float64, 4: 16
0    0.922414
1    0.258216
dtype: float64, 5: 16
0    0.844828
1    0.549296
dtype: float64, 6: 16
0    0.336207
1    0.723005
dtype: float64, 7: 16
0    0.206897
1    0.788732
dtype: float64, 8: 16
0    0.163793
1    0.718310
dtype: float64, 9: 16
0    0.560345
1    0.497653
dtype: float64, 10: 16
0    0.155172
1    0.521127
dtype: float64, 11: 16
0    0.827586
1    0.173709
dtype: float64, 12: 16
0    0.827586
1    0.328638
dtype: float64, 13: 16
0    0.965517
1    0.408451
dtype: float64, 14: 16
0    0.129310
1    0.610329
dtype: float64, 15: 16
0    0.715517
1    0.849765
dtype: float64}
